# Environment

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import random
import os
import settings as settings

# Documentation

In [ ]:
#---------------------------------------------DOCUMENTATION---------------------------------------------------------------------------------------------------------------------------#

# Function Creation:
#### Define a function for data shifting to ensure proper transformation and anonymization.

# Data Loading:
#### keys_df: DataFrame linking original patient IDs to new anonymized keys for de-identification purposes.
#### metadata_keys: DataFrame containing metadata about DICOM files, including PHI (Protected Health Information) for anonymization and system folder implementation in source images.

# Data Filtering & Cleaning:
#### Filter the 'BodyPartExamined' column to select labels of interest.
#### Clean minor inconsistencies across columns.

# Data Anonymization:
#### Create anonymized columns to ensure all patient-identifiable information is properly anonymized according to ethical guidelines and HIPAA's identifying categories.

# Quality Control & Statistics:
#### Extract numbers for the final DataFrame.
#### Provide counts for columns.

# Variable Re-encoding:
#### Translate Spanish values to English.

# Selection of deidentified columns:
#### Layout the final metadata DataFrame.

# Exportation:
#### Save the final tables.

# Function Creation

In [ ]:
var_order = ['PatientID', 'patient_deid', 'patient_group_folder', 'patient_deid_folder', 'Segment', 'Segment_deid', 'SOPInstanceUID', 'SOPInstanceUID_deid', 
             'StudyDate', 'StudyDate_deid', 'StudyTime', 'StudyTime_format', 'Modality', 'StudyDescription', 'BodyPartExamined', 'BodyPart', 
             'CodeValue', 'CodeMeaning',
              'ViewPosition', 'PatientPosition', 'SpatialResolution'] 

def data_shifting(df:pd.DataFrame):
    df = df.sort_values(['PatientID', 'StudyDate', 'StudyTime', 'SOPInstanceUID']).reset_index(drop=True)

    # Column 'patient_deid_folder': format 'p + patient_deid'
    df['patient_deid_folder'] = df.apply(lambda row: 'p' + str(row['patient_deid']), axis=1)
    # Column 'patient_group_folder': format 'p + 2 first numbers of patient_deid'
    df['patient_group_folder'] = df['patient_deid_folder'].str.slice(0, 3)

    # Column 'StudyDate': add multiple weeks for deidentification
    df['StudyDate'] = pd.to_datetime(df['StudyDate'], format='%Y%m%d')
    df['StudyDate_deid'] = 'omitted due to privacy concerns'

    # Column 'StudyTime': convert decimal to integer to remove decimals and transform to time format
    df['StudyTime_format'] = pd.to_datetime(df['StudyTime'].astype(int).astype(str).str.zfill(6), format='%H%M%S').dt.time

    # Column 'Segment_deid': format 's + segment_map', mapping each unique value of "Segment" to a random unique 8-digit number. 
    unique_segments = df['Segment'].unique()
    unique_segments = list(unique_segments) 
    num_segments = len(unique_segments)
    unique_numbers = 'omitted due to privacy concerns'
    segment_mapping = {segment: unique_numbers[i] for i, segment in enumerate(unique_segments)}
    df['segment_map'] = df['Segment'].map(segment_mapping)
    df['Segment_deid'] = df.apply(lambda row: 's' + str(row['segment_map']), axis=1)

    # Column 'SOPInstanceUID_deid': format 'segment_map + _ + ascendent 4 digit number' 
    df['count'] = df.groupby('Segment_deid').cumcount() + 1 
    df['count'] = df['count'].apply(lambda x: f'{x:04d}') 
    df['SOPInstanceUID_deid'] = df['segment_map'].astype(str) + '_' + df['count'] 

    col_reindex = [col for col in var_order if col in df.columns]
    df = df.reindex(columns = col_reindex)

    return df

# New column index and names for final df
var_index = ['patient_group_folder', 'patient_deid', 'patient_deid_folder', 'Segment_deid', 'SOPInstanceUID_deid',  
             'StudyDate_deid', 'StudyTime_format', 'Modality', 'StudyDescription', 'BodyPart',
             'CodeValue', 'CodeMeaning',
             'ViewPosition', 'PatientPosition', 'SpatialResolution'] 

new_names = {
    'patient_group_folder': 'patient_group_folder_id',
    'patient_deid': 'patient_id',
    'patient_deid_folder': 'patient_folder_id',
    'Segment_deid': 'study_id',
    'SOPInstanceUID_deid': 'image_id',
    'StudyDate_deid': 'StudyDate',
    'StudyTime_format': 'StudyTime',
}

# Data Loading


In [ ]:
# Source path and names
source_path = settings.source_path_metadata
file_name = "dicom_data.csv"
keys_df_name = "keys_df.csv"

# Load file
df_images = pd.read_csv(os.path.join(source_path, "source_tables", file_name))
df_images.columns = df_images.columns.str.replace(r'\s+', '', regex=True) #remove blank spaces from column names
print(df_images.info())
print("Nº of rows of actual metadata file:",df_images.size)

# Load keys_df
keys_df = pd.read_csv(os.path.join(source_path, "source_tables", keys_df_name), names=['PatientID', 'patient_deid', 'week_multiples'])
print(keys_df.info())

df_images.head()

# Data Filtering & Cleaning

In [ ]:
# Data Filtering
df_images_cleaned = df_images.copy()

## Remove rows with 'Modality' different than X-ray (CR, DX) and CT (KO and PR)
print("Countings of 'Modality' in raw table:", df_images_cleaned['Modality'].value_counts(dropna=False))
df_images_cleaned = df_images_cleaned[df_images_cleaned['Modality'].isin(['CR', 'DX', 'CT'])]
print("\nQuality Control: Countings of 'Modality' after selecting only X-ray and CT:", 
      df_images_cleaned['Modality'].value_counts(dropna=False))

## Column StudyDescription for different 'Modality'
## X-rays (CR, DX) in 'Modality'
print("\nUnique values of StudyDescription:", df_images_cleaned['StudyDescription'].value_counts(dropna=False))
print("\nUnique values of StudyDescription for X-rays (CR and DX Modalities):",
      df_images_cleaned[df_images_cleaned['Modality'].isin(['CR', 'DX'])]['StudyDescription'].value_counts(dropna=False))
## Renaming incorrect values of StudyDescription for X-rays (CR and DX Modalities)
df_images_cleaned['StudyDescription'] = np.where(
    (df_images_cleaned['Modality'].isin(['CR', 'DX'])) & 
    (df_images_cleaned['StudyDescription'].isin(['TC TORAX SIN CONTRASTE', 'TC ARTERIAS PULMONARES'])), 
    df_images_cleaned['Modality'], 
    df_images_cleaned['StudyDescription'])
print("\nQuality Control: Unique values of StudyDescription for X-rays (CR and DX Modalities) after data cleaning:",
      df_images_cleaned[df_images_cleaned['Modality'].isin(['CR', 'DX'])]['StudyDescription'].value_counts(dropna=False))
## CT scans (CT) in 'Modality'
print("\nUnique values of StudyDescription for CT Modalities:", 
      df_images_cleaned[df_images_cleaned['Modality'].isin(['CT'])]['StudyDescription'].value_counts(dropna=False))
## Renaming incorrect values of StudyDescription for X-rays (CR and DX Modalities)
df_images_cleaned['StudyDescription'] = np.where(
    (df_images_cleaned['Modality'].isin(['CT'])) & 
    (df_images_cleaned['StudyDescription'].isin(['RX TORAX AP'])), 
    df_images_cleaned['Modality'], 
    df_images_cleaned['StudyDescription']
)
print("\nQuality Control: Unique values of StudyDescription for CT Modalities after data cleaning:", 
      df_images_cleaned[df_images_cleaned['Modality'].isin(['CT'])]['StudyDescription'].value_counts(dropna=False))

## Column 'BodyPartExamined'
print("\nUnique values of 'BodyPartExamined':", df_images_cleaned['BodyPartExamined'].value_counts(dropna=False))
## Filter rows where 'BodyPartExamined' is 'Unknown'
unknown_filter = df_images_cleaned['BodyPartExamined'].isin(['Unknown'])
## Replace Unknown values with values from 'StudyDescription'
df_images_cleaned['BodyPartExamined']  = df_images_cleaned['BodyPartExamined'].replace('Unknown', np.nan).fillna(df_images_cleaned['StudyDescription'])
## Count new values of previous 'Unknown' values
print("\nUnique values of 'BodyPartExamined' originally 'Unknown':", df_images_cleaned[unknown_filter]['BodyPartExamined'].value_counts(dropna=False))
del unknown_filter

## Columns 'CodeValue' and 'CodeMeaning'
print("\nUnique values of 'CodeValue:", df_images_cleaned['CodeValue'].value_counts(dropna=False))
print("\nUnique values of 'CodeMeaning:", df_images_cleaned['CodeMeaning'].value_counts(dropna=False))
## Removing CodeMeaning with values outside from Chest/Thorax
df_images_cleaned = df_images_cleaned[~df_images_cleaned['CodeMeaning'].isin(['TC CRANEO SIN CONTRASTE'])]
print("\nQuality Control: Unique values of 'CodeMeaning:", df_images_cleaned['CodeMeaning'].value_counts(dropna=False))
## Unique values of 'CodeMeaning' for X-ray and CT modalities
print("\nUnique values of CodeMeaning for X-rays (CR and DX Modalities):", 
       df_images_cleaned[df_images_cleaned['Modality'].isin(['CR', 'DX'])]['CodeMeaning'].value_counts(dropna=False))
print("\nUnique values of CodeMeaning for CT Modalities:", 
      df_images_cleaned[df_images_cleaned['Modality'].isin(['CT'])]['CodeMeaning'].value_counts(dropna=False))
## Unique values of 'StudyDescription' for NaN values in CodeMeaning
CodeMeaning_NaN = df_images_cleaned[df_images_cleaned['CodeMeaning'].isna()]
print("\nUnique values of 'StudyDescription' for NaN values in 'CodeMeaning':", CodeMeaning_NaN['StudyDescription'].value_counts(dropna=False))
del CodeMeaning_NaN

## Lists of values containing Thorax/Chest to filter in the 'BodyPartExamined' column
pulmonary_arteries = [
    'TC ARTERIAS PULMONARES', 'ANGIO TORACICO', 'CR Y TEP SUSTR', 'TC  TEP', 'TC  TEP SUSTR',
    'TC SIN Y TEP', 'TC T EP SUSTR', 'TC TEP', 'TC TEP SUST', 'TC TEP SUSTR',
    'TC TEPN', 'TEP', 'TEP SUSTR']

thorax_chest = [
    'TORAX', 'LUNG', 'PECHO', 'TACAR', 'TC TACAR', 'TC TORAX', 
    'TC TORAX AR', 'TC TORAX BD', 'TC TX', 
    'TC TX BD', 'TORAX  BD', 'TORAX AR', 'TORAX BD', 'TX', 'TX BD', 'TX SC', 'CHEST', 
    'RX TORAX AP', 'RX TORAX AP Y LATERAL']

body = [
    'TC TORACO-ABDOMINO-PELVICO CON CONTRASTE', 
    'TC TORACO-ABDOMINO-PELVICO SIN CONTRASTE', 'TC TORAX BAJA DOSIS', 'TC TORAX CON CONTRASTE', 
    'TC TORAX DE ALTA DEFINICION', 'TC TORAX SIN CONTRASTE', 'TAP', 'TC TAP', 'TORAX-ABD-PELVIS', 
    'TORAX-ABDOMEN', 'TORAXABDPELV']


## Filter the 'BodyPartExamined' column with the values from the previous lists
df_images_cleaned = df_images_cleaned[df_images_cleaned['BodyPartExamined'].isin(pulmonary_arteries + thorax_chest + body)]
df_images_cleaned['BodyPart'] = np.where(df_images_cleaned['BodyPartExamined'].isin(pulmonary_arteries), 'Pulmonary arteries',
                            np.where(df_images_cleaned['BodyPartExamined'].isin(thorax_chest), 'Thorax/Chest',
                                     np.where(df_images_cleaned['BodyPartExamined'].isin(body), 'Body', 'Other')))

# Get the count of unique values and sort them for 'BodyPart'
print("\nUnique values of 'BodyPart'", df_images_cleaned['BodyPart'].value_counts(dropna=False))

# Data Anonimization

In [ ]:
# Merging df_images_cleaned with keys_df
metadata_keys = df_images_cleaned.merge(keys_df, how='left', on='PatientID')
print("\n",metadata_keys.info())

# Data Shifting
metadata_keys = data_shifting(metadata_keys)

print(metadata_keys.info())
metadata_keys.head()

# Quality Control & Statistics

In [ ]:
# Overall: unique number of patients, folders and subfolders
print("\nUnique patients with images are:", metadata_keys['patient_deid'].nunique())
print("Unique study subfolders are:", metadata_keys['Segment_deid'].nunique())
print("Unique images are:", metadata_keys['SOPInstanceUID_deid'].nunique())
print("Unique CT are:", metadata_keys[metadata_keys['Modality'].isin(['CT'])]['SOPInstanceUID_deid'].nunique())
print("Unique X-rays are:", metadata_keys[metadata_keys['Modality'].isin(['CR', 'DX'])]['SOPInstanceUID_deid'].nunique())

## Nº of studies containing both CT and X-ray images
study_id_with_CT = metadata_keys[metadata_keys['Modality'] == 'CT']['Segment'].unique() # Filter study_id containing CT images
study_id_with_DX_CR = metadata_keys[metadata_keys['Modality'].isin(['DX', 'CR'])]['Segment'].unique() # Filter study_id containing DX or CR images
study_id_with_both = set(study_id_with_CT) & set(study_id_with_DX_CR) # Find study_id that are in both sets
print("\nUnique number of studies containing both CT and DX/CR:", len(study_id_with_both), "-->specifically study_id:", study_id_with_both)
del (study_id_with_CT, study_id_with_DX_CR, study_id_with_both)

# Overall: unique number of patients in keys_df and metadata_keys
#3 Nº of unique de-identified patient identifiers in keys_df
print("\nUnique patient_id numbers in keys table are", keys_df['PatientID'].nunique())
print("Unique patient_deid numbers in keys table are", keys_df['patient_deid'].nunique())
## Nº of unique de-identified patient identifiers in metadata_keys
print("Unique patient_id numbers in metadata_keys are", metadata_keys['PatientID'].nunique())
print("Unique patient_deid numbers in metadata_keys table are", metadata_keys['patient_deid'].nunique())

# Column 'Modality'
print("\nUnique values of Modality:", metadata_keys['Modality'].value_counts(dropna=False))

# Column 'StudyDescription'
print("\nUnique values of StudyDescription:", metadata_keys['StudyDescription'].value_counts(dropna=False))
print("\nUnique values of StudyDescription for CR and DX Modalities:",
       metadata_keys[metadata_keys['Modality'].isin(['CR', 'DX'])]['StudyDescription'].value_counts(dropna=False))
print("\nUnique values of SpatialResolution for CT Modalities:",
       metadata_keys[metadata_keys['Modality'].isin(['CT'])]['StudyDescription'].value_counts(dropna=False))

# Column 'CodeMeaning'
print("\nUnique values of CodeMeaning:", metadata_keys['CodeMeaning'].value_counts(dropna=False))
print("\nUnique values of CodeMeaning for CR and DX Modalities:",
       metadata_keys[metadata_keys['Modality'].isin(['CR', 'DX'])]['CodeMeaning'].value_counts(dropna=False))
print("\nUnique values of CodeMeaning for CT Modalities:",
       metadata_keys[metadata_keys['Modality'].isin(['CT'])]['CodeMeaning'].value_counts(dropna=False))

# Column 'ViewPosition'
print("\nUnique values of ViewPosition:", metadata_keys['ViewPosition'].value_counts(dropna=False))
print("\nUnique values of ViewPosition for CT Modalities:",
       metadata_keys[metadata_keys['Modality'].isin(['CT'])]['ViewPosition'].value_counts(dropna=False))
print("\nUnique values of ViewPosition for CR and DX Modalities:",
       metadata_keys[metadata_keys['Modality'].isin(['CR', 'DX'])]['ViewPosition'].value_counts(dropna=False))

# Column 'PatientPosition'
print("\nUnique values of PatientPosition:", metadata_keys['PatientPosition'].value_counts(dropna=False))
print("\nUnique values of PatientPosition for CR and DX Modalities:",
       metadata_keys[metadata_keys['Modality'].isin(['CR', 'DX'])]['PatientPosition'].value_counts(dropna=False))
print("\nUnique values of PatientPosition for CT Modalities:",
       metadata_keys[metadata_keys['Modality'].isin(['CT'])]['PatientPosition'].value_counts(dropna=False))

# Column 'SpatialResolution'
print("\nUnique values of SpatialResolution:", metadata_keys['SpatialResolution'].value_counts(dropna=False))
print("\nUnique values of SpatialResolution for CR and DX Modalities:",
       metadata_keys[metadata_keys['Modality'].isin(['CR', 'DX'])]['SpatialResolution'].value_counts(dropna=False))
print("\nUnique values of SpatialResolution for CT Modalities:",
       metadata_keys[metadata_keys['Modality'].isin(['CT'])]['SpatialResolution'].value_counts(dropna=False))

# Variable Re-encoding

In [ ]:
# Variable Re-encoding
metadata_keys_reencoded = metadata_keys.copy()

study_description_translation = {
    "TC ARTERIAS PULMONARES": "CT Pulmonary Arteries",
    "TC TORAX SIN CONTRASTE": "CT Chest without contrast",
    "TC TORAX BAJA DOSIS": "CT Chest low-dose",
    "TC TORACO-ABDOMINO-PELVICO CON CONTRASTE": "CT Thoraco-Abdomino-Pelvic with contrast",
    "TC TORAX CON CONTRASTE": "CT Chest with contrast",
    "TC TORAX DE ALTA DEFINICION": "CT Chest High-Resolution",
    "TC TORACO-ABOMINO-PELVICO SIN CONTRASTE": "CT Thoraco-Abdomino-Pelvic without contrast",
    "RX TORAX AP": "Radiograph Chest",
    "CT": "CT",
    "RX TORAX AP Y LATERAL": "Radiograph Chest",
    "CR": "Radiograph Chest",
    "RX Simple": "Radiograph Chest",
    "RX SIMPLE": "Radiograph Chest"
}

code_meaning_translation = {
    'TC ARTERIAS PULMONARES': 'CT Pulmonary Arteries',
    'TAC': 'CT',
    'TC TORAX DE ALTA DEFINICION': 'CT Chest High-Resolution',
    'TC TORAX SIN CONTRASTE': 'CT Chest without contrast',
    'TC TORAX CON CONTRASTE': 'CT Chest with contrast',
    'TC TORACO-ABDOMINO-PELVICO CON CONTRASTE': 'CT Thoraco-Abdomino-Pelvic with contrast',
    'TC': 'CT',
    'TC TORAX BAJA DOSIS': 'CT Chest low-dose',
    'RX Simple': 'X-ray simple',
    'TC TORACO-ABOMINO-PELVICO SIN CONTRASTE': 'CT Thoraco-Abdomino-Pelvic without contrast',
    'RX TORAX AP': 'X-ray chest AP',
    'RX SIMPLE': 'X-ray simple',
    'RX TORAX AP Y LATERAL': 'X-ray chest AP and lateral',
    'RX GENERAL': 'X-ray General',
}

# Apply the translation to the specific columns
metadata_keys_reencoded['StudyDescription'] = metadata_keys_reencoded['StudyDescription'].map(study_description_translation)
print("\nUnique values of StudyDescription:", metadata_keys_reencoded['StudyDescription'].value_counts(dropna=False))
metadata_keys_reencoded['CodeMeaning'] = metadata_keys_reencoded['CodeMeaning'].map(code_meaning_translation)
print("\nUnique values of CodeMeaning:", metadata_keys_reencoded['CodeMeaning'].value_counts(dropna=False))

metadata_keys_reencoded = metadata_keys_reencoded.sort_values(['patient_deid', 'StudyDate_deid', 'StudyTime_format', 'SOPInstanceUID_deid']).reset_index(drop=True)

# Selection of deidentified columns

In [ ]:
# Selection of required columns for Physionet metadata table
metadata_physionet = metadata_keys_reencoded.copy()
col_reindex = [col for col in var_index if col in metadata_keys.columns]
metadata_physionet = metadata_physionet.reindex(columns = col_reindex)
metadata_physionet.rename(columns=new_names, inplace=True)
metadata_physionet = metadata_physionet.sort_values(['patient_id', 'StudyDate', 'StudyTime', 'image_id']).reset_index(drop=True)
print(metadata_physionet.info())

# Exportation

In [ ]:
# Export csv
metadata_keys.to_csv(os.path.join(source_path, "output_tables", "metadata_keys.csv"), sep = ',', index=False)
metadata_keys_reencoded.to_csv(os.path.join(source_path, "output_tables", "metadata_keys_reencoded.csv"), sep = ',', index=False)
metadata_physionet.to_csv(os.path.join(source_path, "output_tables", "metadata_physionet.csv"), sep = ',', index=False)